In [1]:
import pandas as pd
import re

## スペースの有無のチェック

In [40]:
# 前処理関数
def preprocess_company_name_space(name):
    # 全角スペースを半角スペースに変換し、スペースを削除
    name = re.sub(r'\s+', '', name.replace('　', ' '))
    return name

# データセットのロード
partner_center_df = pd.read_csv('dataset/partner_center/partner_center_space.csv')
customer_book_df = pd.read_csv('dataset/customer_book/customer_book_space.csv')

# 比較結果を保存するためのリスト
comparison_results = []

# 比較ループ
for _, pc_row in partner_center_df.iterrows():
    pc_name = preprocess_company_name(pc_row['会社名'])
    for _, cb_row in customer_book_df.iterrows():
        cb_name = preprocess_company_name_space(cb_row['会社名'])
        if pc_row['サブスクリプションID'] == cb_row['サブスクリプションID']:
            result = {
                "partner center 会社名": pc_row['会社名'],
                "customer book 会社名": cb_row['会社名'],
                "サブスクリプションID": pc_row['サブスクリプションID'],
                "妥当性": pc_name == cb_name
            }
            comparison_results.append(result)

comparison_df = pd.DataFrame(comparison_results)

# 比較結果のCSVファイルを保存
comparison_df.to_csv('result/comparison_results_space.csv', index=False)

comparison_df

,partner center 会社名,customer book 会社名,サブスクリプションID,妥当性
0,株式会社九州電子,株式会社 九州電子,1111-1111,True
1,株式会社ICテクノロジーズ,株式会社 IC テクノロジーズ,1111-1112,True
2,株式会社夢組,株式会社 夢組,1111-1113,True
3,株式会社ソフトウェアサービス,株式会社ソフト ウェア サービス,1111-1114,True
4,株式会社イーシステム,株式会社 イーシステム,1111-1115,True
5,株式会社ニューウェーブ,株式会社ニュー ウェーブ,1111-1116,True
6,株式会社アーステック,株式会社 アース テック,1111-1117,True
7,株式会社サンライズ,株式会社サン ライズ,1111-1118,True
8,株式会社グリーンテック,株式会社ブルースカイテック,1111-1119,False
9,株式会社サンテック,株式会社サンライズテック,1111-1120,False


In [47]:
# 妥当性チェック結果と正解データの比較
comparison_results_df = pd.DataFrame(comparison_results)

# 正解データセットのロード
expected_validity_df = pd.read_csv('dataset/correct_data/comparison_results_with_validity_space.csv')

# サブスクリプションIDをキーとしてマージして比較
comparison_with_truth_df = comparison_results_df.merge(
    expected_validity_df, 
    on=["partner center 会社名", "customer book 会社名", "サブスクリプションID"], 
    suffixes=('_predicted', '_true')
)

# 妥当性が一致しているかどうかを確認
comparison_with_truth_df['一致'] = comparison_with_truth_df['妥当性_predicted'] == comparison_with_truth_df['妥当性_true']

comparison_with_truth_df = comparison_with_truth_df[[
    'partner center 会社名', 'customer book 会社名', 
    'サブスクリプションID', 'M365 プラン', '契約名', 
    '妥当性_predicted', '妥当性_true', '一致'
]]

# 比較結果の表示
comparison_with_truth_df


,partner center 会社名,customer book 会社名,サブスクリプションID,M365 プラン,契約名,妥当性_predicted,妥当性_true,一致
0,株式会社九州電子,株式会社 九州電子,1111-1111,M365 Business Basic,契約A,True,True,True
1,株式会社ICテクノロジーズ,株式会社 IC テクノロジーズ,1111-1112,M365 Business Basic,契約B,True,True,True
2,株式会社夢組,株式会社 夢組,1111-1113,M365 Business Basic,契約C,True,True,True
3,株式会社ソフトウェアサービス,株式会社ソフト ウェア サービス,1111-1114,M365 Business Basic,契約D,True,True,True
4,株式会社イーシステム,株式会社 イーシステム,1111-1115,M365 Business Basic,契約E,True,True,True
5,株式会社ニューウェーブ,株式会社ニュー ウェーブ,1111-1116,M365 Business Basic,契約F,True,True,True
6,株式会社アーステック,株式会社 アース テック,1111-1117,M365 Business Basic,契約G,True,True,True
7,株式会社サンライズ,株式会社サン ライズ,1111-1118,M365 Business Basic,契約H,True,True,True
8,株式会社グリーンテック,株式会社ブルースカイテック,1111-1119,M365 Business Basic,契約I,False,False,True
9,株式会社サンテック,株式会社サンライズテック,1111-1120,M365 Business Basic,契約J,False,False,True


## 半角全角有無

In [59]:
# 妥当性チェックのための関数
def convert_to_fullwidth(text):
    # 半角カタカナと全角カタカナの対応辞書
    kana_map = {
        'ｶ': 'カ', 'ｷ': 'キ', 'ｸ': 'ク', 'ｹ': 'ケ', 'ｺ': 'コ',
        'ｻ': 'サ', 'ｼ': 'シ', 'ｽ': 'ス', 'ｾ': 'セ', 'ｿ': 'ソ',
        'ﾀ': 'タ', 'ﾁ': 'チ', 'ﾂ': 'ツ', 'ﾃ': 'テ', 'ﾄ': 'ト',
        'ﾊ': 'ハ', 'ﾋ': 'ヒ', 'ﾌ': 'フ', 'ﾍ': 'ヘ', 'ﾎ': 'ホ',
        'ｳ': 'ウ', 'ｦ': 'ヲ', 'ﾜ': 'ワ', 'ｲ': 'イ', 'ｴ': 'エ', 'ｵ': 'オ',
        'ｶﾞ': 'ガ', 'ｷﾞ': 'ギ', 'ｸﾞ': 'グ', 'ｹﾞ': 'ゲ', 'ｺﾞ': 'ゴ',
        'ｻﾞ': 'ザ', 'ｼﾞ': 'ジ', 'ｽﾞ': 'ズ', 'ｾﾞ': 'ゼ', 'ｿﾞ': 'ゾ',
        'ﾀﾞ': 'ダ', 'ﾁﾞ': 'ヂ', 'ﾂﾞ': 'ヅ', 'ﾃﾞ': 'デ', 'ﾄﾞ': 'ド',
        'ﾊﾞ': 'バ', 'ﾋﾞ': 'ビ', 'ﾌﾞ': 'ブ', 'ﾍﾞ': 'ベ', 'ﾎﾞ': 'ボ',
        'ﾊﾟ': 'パ', 'ﾋﾟ': 'ピ', 'ﾌﾟ': 'プ', 'ﾍﾟ': 'ペ', 'ﾎﾟ': 'ポ',
        'ｱ': 'ア', 'ｲ': 'イ', 'ｳ': 'ウ', 'ｴ': 'エ', 'ｵ': 'オ',
        'ｧ': 'ァ', 'ｨ': 'ィ', 'ｩ': 'ゥ', 'ｪ': 'ェ', 'ｫ': 'ォ',
        'ｳﾞ': 'ヴ', 'ﾔ': 'ヤ', 'ﾕ': 'ユ', 'ﾖ': 'ヨ', 'ｬ': 'ャ', 'ｭ': 'ュ', 'ｮ': 'ョ',
        'ﾏ': 'マ', 'ﾐ': 'ミ', 'ﾑ': 'ム', 'ﾒ': 'メ', 'ﾓ': 'モ',
        'ﾅ': 'ナ', 'ﾆ': 'ニ', 'ﾇ': 'ヌ', 'ﾈ': 'ネ', 'ﾉ': 'ノ',
        'ﾗ': 'ラ', 'ﾘ': 'リ', 'ﾙ': 'ル', 'ﾚ': 'レ', 'ﾛ': 'ロ',
        'ﾜ': 'ワ', 'ﾝ': 'ン', 'ｰ': 'ー', 'ﾝ': 'ン', '｡': '。', '､': '、',
        '｢': '「', '｣': '」', '･': '・'
    }

    # 濁点・半濁点の辞書
    dakuten_map = {
        'カ': 'ガ', 'キ': 'ギ', 'ク': 'グ', 'ケ': 'ゲ', 'コ': 'ゴ',
        'サ': 'ザ', 'シ': 'ジ', 'ス': 'ズ', 'セ': 'ゼ', 'ソ': 'ゾ',
        'タ': 'ダ', 'チ': 'ヂ', 'ツ': 'ヅ', 'テ': 'デ', 'ト': 'ド',
        'ハ': 'バ', 'ヒ': 'ビ', 'フ': 'ブ', 'ヘ': 'ベ', 'ホ': 'ボ',
        'ウ': 'ヴ'
    }
    
    handakuten_map = {
        'ハ': 'パ', 'ヒ': 'ピ', 'フ': 'プ', 'ヘ': 'ペ', 'ホ': 'ポ'
    }

    # 変換処理
    result = ''
    i = 0
    while i < len(text):
        if i + 1 < len(text) and text[i + 1] in ('ﾞ', 'ﾟ'):
            char = kana_map.get(text[i], text[i])
            if text[i + 1] == 'ﾞ':  # 濁点
                result += dakuten_map.get(char, char)
            elif text[i + 1] == 'ﾟ':  # 半濁点
                result += handakuten_map.get(char, char)
            i += 2
        else:
            result += kana_map.get(text[i], text[i])
            i += 1

    return result
    
def preprocess_company_name_half_letter(name):
    # 全角スペースを半角スペースに変換し、スペースを削除
    name = re.sub(r'\s+', '', name.replace('　', ' '))
    # 半角カタカナを全角カタカナに変換
    name = name.translate(str.maketrans({
        'Ａ': 'A', 'Ｂ': 'B', 'Ｃ': 'C', 'Ｄ': 'D', 'Ｅ': 'E', 'Ｆ': 'F', 'Ｇ': 'G', 'Ｈ': 'H', 'Ｉ': 'I', 'Ｊ': 'J',
        'Ｋ': 'K', 'Ｌ': 'L', 'Ｍ': 'M', 'Ｎ': 'N', 'Ｏ': 'O', 'Ｐ': 'P', 'Ｑ': 'Q', 'Ｒ': 'R', 'Ｓ': 'S', 'Ｔ': 'T',
        'Ｕ': 'U', 'Ｖ': 'V', 'Ｗ': 'W', 'Ｘ': 'X', 'Ｙ': 'Y', 'Ｚ': 'Z', '１': '1', '２': '2', '３': '3', '４': '4',
        '５': '5', '６': '6', '７': '7', '８': '8', '９': '9', '０': '0',
        'ａ': 'a', 'ｂ': 'b', 'ｃ': 'c', 'ｄ': 'd', 'ｅ': 'e', 'ｆ': 'f', 'ｇ': 'g', 'ｈ': 'h', 'ｉ': 'i', 'ｊ': 'j',
        'ｋ': 'k', 'ｌ': 'l', 'ｍ': 'm', 'ｎ': 'n', 'ｏ': 'o', 'ｐ': 'p', 'ｑ': 'q', 'ｒ': 'r', 'ｓ': 's', 'ｔ': 't',
        'ｕ': 'u', 'ｖ': 'v', 'ｗ': 'w', 'ｘ': 'x', 'ｙ': 'y', 'ｚ': 'z'
    }))
    # 半角カタカナを全角カタカナに変換
    name = name.translate(str.maketrans({
        'ｱ': 'ア', 'ｲ': 'イ', 'ｳ': 'ウ', 'ｴ': 'エ', 'ｵ': 'オ',
        'ｶ': 'カ', 'ｷ': 'キ', 'ｸ': 'ク', 'ｹ': 'ケ', 'ｺ': 'コ',
        'ｻ': 'サ', 'ｼ': 'シ', 'ｽ': 'ス', 'ｾ': 'セ', 'ｿ': 'ソ',
        'ﾀ': 'タ', 'ﾁ': 'チ', 'ﾂ': 'ツ', 'ﾃ': 'テ', 'ﾄ': 'ト',
        'ﾅ': 'ナ', 'ﾆ': 'ニ', 'ﾇ': 'ヌ', 'ﾈ': 'ネ', 'ﾉ': 'ノ',
        'ﾊ': 'ハ', 'ﾋ': 'ヒ', 'ﾌ': 'フ', 'ﾍ': 'ヘ', 'ﾎ': 'ホ',
        'ﾏ': 'マ', 'ﾐ': 'ミ', 'ﾑ': 'ム', 'ﾒ': 'メ', 'ﾓ': 'モ',
        'ﾔ': 'ヤ', 'ﾕ': 'ユ', 'ﾖ': 'ヨ',
        'ﾗ': 'ラ', 'ﾘ': 'リ', 'ﾙ': 'ル', 'ﾚ': 'レ', 'ﾛ': 'ロ',
        'ﾜ': 'ワ', 'ｦ': 'ヲ', 'ﾝ': 'ン',
        'ｧ': 'ァ', 'ｨ': 'ィ', 'ｩ': 'ゥ', 'ｪ': 'ェ', 'ｫ': 'ォ',
        'ｬ': 'ャ', 'ｭ': 'ュ', 'ｮ': 'ョ',
        'ｯ': 'ッ', 'ｰ': 'ー'
    }))
    name = convert_to_fullwidth(name)
    # 英字を大文字に変換
    name = name.upper()
    return name

# データセットのロード
partner_center_df = pd.read_csv('dataset/partner_center/partner_center_halfletter.csv')
customer_book_df = pd.read_csv('dataset/customer_book/customer_book_halfletter.csv')

# 比較結果を保存するためのリスト
comparison_results = []

# 比較ループ
for _, pc_row in partner_center_df.iterrows():
    pc_name = preprocess_company_name_half_letter(pc_row['会社名'])
    print(pc_name)
    for _, cb_row in customer_book_df.iterrows():
        cb_name = preprocess_company_name_half_letter(cb_row['会社名'])
        if pc_row['サブスクリプションID'] == cb_row['サブスクリプションID']:
            result = {
                "partner center 会社名": pc_row['会社名'],
                "customer book 会社名": cb_row['会社名'],
                "サブスクリプションID": pc_row['サブスクリプションID'],
                "妥当性": pc_name == cb_name
            }
            comparison_results.append(result)

comparison_df = pd.DataFrame(comparison_results)

合同会社ALLRIGHT
株式会社テクノロジーズ123
株式会社TECHNOLOGY
株式会社PREMIUMSERVICE
株式会社インターネット
株式会社グローバルウェーブ
株式会社アドバンス
株式会社ST092
株式会社SILVERBLACKS
株式会社サンセット


In [60]:
comparison_df

,partner center 会社名,customer book 会社名,サブスクリプションID,妥当性
0,合同会社ＡｌｌＲｉｇｈｔ,合同会社AllRight,2222-1111,True
1,株式会社テクノロジーズ１２３,株式会社テクノロジーズ123,2222-1112,True
2,株式会社Ｔｅｃｈｎｏｌｏｇｙ,株式会社Technology,2222-1113,True
3,株式会社ＰＲＥＭＩＵＭ ＳＥＲＶＩＣＥ,株式会社PREMIUMSERVICE,2222-1114,True
4,株式会社ｲﾝﾀｰﾈｯﾄ,株式会社インターネット,2222-1115,True
5,株式会社ｸﾞﾛｰﾊﾞﾙｳｪｰﾌﾞ,株式会社グローバルウェーブ,2222-1116,True
6,株式会社アドバンス,株式会社アドバンス,2222-1117,True
7,株式会社ｓｔ０９２,株式会社st092,2222-1118,True
8,株式会社Ｓｉｌｖｅｒ Ｂｌａｃｋs,株式会社Silver Black,2222-1119,False
9,株式会社サンセット,株式会社サンセットテック,2222-1120,False


In [61]:
# 妥当性チェック結果と正解データの比較
comparison_results_df = pd.DataFrame(comparison_results)

# 正解データセットのロード
expected_validity_df = pd.read_csv('dataset/correct_data/comparison_results_with_validity_halfletter.csv')

expected_validity_df


,partner center 会社名,customer book 会社名,サブスクリプションID,M365 プラン,契約名,妥当性
0,合同会社ＡｌｌＲｉｇｈｔ,合同会社AllRight,2222-1111,M365 Business Standard,契約A,True
1,株式会社テクノロジーズ１２３,株式会社テクノロジーズ123,2222-1112,M365 Business Standard,契約B,True
2,株式会社Ｔｅｃｈｎｏｌｏｇｙ,株式会社Technology,2222-1113,M365 Business Standard,契約C,True
3,株式会社ＰＲＥＭＩＵＭ ＳＥＲＶＩＣＥ,株式会社PREMIUMSERVICE,2222-1114,M365 Business Standard,契約D,True
4,株式会社ｲﾝﾀｰﾈｯﾄ,株式会社インターネット,2222-1115,M365 Business Standard,契約E,True
5,株式会社ｸﾞﾛｰﾊﾞﾙｳｪｰﾌﾞ,株式会社グローバルウェーブ,2222-1116,M365 Business Standard,契約F,True
6,株式会社アドバンス,株式会社アドバンス,2222-1117,M365 Business Standard,契約G,True
7,株式会社ｓｔ０９２,株式会社st092,2222-1118,M365 Business Standard,契約H,True
8,株式会社Ｓｉｌｖｅｒ Ｂｌａｃｋs,株式会社Silver Black,2222-1119,M365 Business Standard,契約I,False
9,株式会社サンセット,株式会社サンセットテック,2222-1120,M365 Business Standard,契約J,False


In [64]:
# サブスクリプションIDをキーとしてマージして比較
comparison_with_truth_df = comparison_results_df.merge(
    expected_validity_df, 
    on=["partner center 会社名", "customer book 会社名", "サブスクリプションID"], 
    suffixes=('_predicted', '_true')
)

# 妥当性が一致しているかどうかを確認
comparison_with_truth_df['一致'] = comparison_with_truth_df['妥当性_predicted'] == comparison_with_truth_df['妥当性_true']

comparison_with_truth_df = comparison_with_truth_df[[
    'partner center 会社名', 'customer book 会社名', 
    'サブスクリプションID', 'M365 プラン', '契約名', 
    '妥当性_predicted', '妥当性_true', '一致'
]]

# 比較結果の表示
comparison_with_truth_df

,partner center 会社名,customer book 会社名,サブスクリプションID,M365 プラン,契約名,妥当性_predicted,妥当性_true,一致
0,合同会社ＡｌｌＲｉｇｈｔ,合同会社AllRight,2222-1111,M365 Business Standard,契約A,True,True,True
1,株式会社テクノロジーズ１２３,株式会社テクノロジーズ123,2222-1112,M365 Business Standard,契約B,True,True,True
2,株式会社Ｔｅｃｈｎｏｌｏｇｙ,株式会社Technology,2222-1113,M365 Business Standard,契約C,True,True,True
3,株式会社ＰＲＥＭＩＵＭ ＳＥＲＶＩＣＥ,株式会社PREMIUMSERVICE,2222-1114,M365 Business Standard,契約D,True,True,True
4,株式会社ｲﾝﾀｰﾈｯﾄ,株式会社インターネット,2222-1115,M365 Business Standard,契約E,True,True,True
5,株式会社ｸﾞﾛｰﾊﾞﾙｳｪｰﾌﾞ,株式会社グローバルウェーブ,2222-1116,M365 Business Standard,契約F,True,True,True
6,株式会社アドバンス,株式会社アドバンス,2222-1117,M365 Business Standard,契約G,True,True,True
7,株式会社ｓｔ０９２,株式会社st092,2222-1118,M365 Business Standard,契約H,True,True,True
8,株式会社Ｓｉｌｖｅｒ Ｂｌａｃｋs,株式会社Silver Black,2222-1119,M365 Business Standard,契約I,False,False,True
9,株式会社サンセット,株式会社サンセットテック,2222-1120,M365 Business Standard,契約J,False,False,True


## 株式会社の有無

In [154]:
# データセットのロード
partner_center_df = pd.read_csv('dataset/partner_center/partner_center_kabushiki.csv')
customer_book_df = pd.read_csv('dataset/customer_book/customer_book_kabushiki.csv')

# 会社名を正規化する関数
def normalize_company_name(name):
    # 法人種別を削除
    name = re.sub(r'(株式会社|合同会社|有限会社)', '', name)
    return name.strip()

# 妥当性チェックの結果を保存するリスト
comparison_results = []

# 比較ループ
for _, pc_row in partner_center_df.iterrows():
    pc_name_normalized = normalize_company_name(pc_row['会社名'])
    for _, cb_row in customer_book_df.iterrows():
        if pc_row['サブスクリプションID'] == cb_row['サブスクリプションID']:
            cb_name_normalized = normalize_company_name(cb_row['会社名'])
            result = {
                "partner center 会社名": pc_row['会社名'],
                "customer book 会社名": cb_row['会社名'],
                "サブスクリプションID": pc_row['サブスクリプションID'],
                "妥当性": pc_name_normalized == cb_name_normalized
            }
            comparison_results.append(result)

# 比較結果のデータフレームを作成
comparison_results_df = pd.DataFrame(comparison_results)

comparison_results_df

,partner center 会社名,customer book 会社名,サブスクリプションID,妥当性
0,T&Dリスクソリューションズ株式会社,T&Dリスクソリューションズ,4444-1111,True
1,株式会社ミツビシ電機,ミツビシ電機,4444-1112,True
2,トヨタ自動車株式会社,トヨタ自動車,4444-1113,True
3,ソニー株式会社,ソニー,4444-1114,True
4,株式会社日立製作所,日立製作所,4444-1115,True
5,パナソニック株式会社,パナソニック,4444-1116,True
6,株式会社シャープ,シャープ,4444-1117,True
7,株式会社ホンダ,ホンダ,4444-1118,True
8,富士通株式会社,日産,4444-1119,False
9,リコー株式会社,リコーテック,4444-1120,False


In [155]:
expected_validity_df = pd.read_csv('dataset/correct_data/comparison_results_with_validity_kabushiki.csv')

# サブスクリプションIDをキーとしてマージして比較
comparison_with_truth_df = comparison_results_df.merge(
    expected_validity_df, 
    on=["partner center 会社名", "customer book 会社名", "サブスクリプションID"], 
    suffixes=('_predicted', '_true')
)

# 妥当性が一致しているかどうかを確認
comparison_with_truth_df['一致'] = comparison_with_truth_df['妥当性_predicted'] == comparison_with_truth_df['妥当性_true']

# カラムの順序を指定して表示
comparison_with_truth_df = comparison_with_truth_df[[
    'partner center 会社名', 'customer book 会社名', 
    'サブスクリプションID', 'M365 プラン', '契約名', 
    '妥当性_predicted', '妥当性_true', '一致'
]]

In [156]:
comparison_with_truth_df

,partner center 会社名,customer book 会社名,サブスクリプションID,M365 プラン,契約名,妥当性_predicted,妥当性_true,一致
0,T&Dリスクソリューションズ株式会社,T&Dリスクソリューションズ,4444-1111,M365 Enterprise E3,契約A,True,True,True
1,株式会社ミツビシ電機,ミツビシ電機,4444-1112,M365 Enterprise E3,契約B,True,True,True
2,トヨタ自動車株式会社,トヨタ自動車,4444-1113,M365 Enterprise E3,契約C,True,True,True
3,ソニー株式会社,ソニー,4444-1114,M365 Enterprise E3,契約D,True,True,True
4,株式会社日立製作所,日立製作所,4444-1115,M365 Enterprise E3,契約E,True,True,True
5,パナソニック株式会社,パナソニック,4444-1116,M365 Enterprise E3,契約F,True,True,True
6,株式会社シャープ,シャープ,4444-1117,M365 Enterprise E3,契約G,True,True,True
7,株式会社ホンダ,ホンダ,4444-1118,M365 Enterprise E3,契約H,True,True,True
8,富士通株式会社,日産,4444-1119,M365 Enterprise E3,契約I,False,False,True
9,リコー株式会社,リコーテック,4444-1120,M365 Enterprise E3,契約J,False,False,True


## カナ/英字、略称、その他

「カナ/英字、略称、その他」に関して会社名の妥当性をチェックするためには、以下のようなアプローチが考えられます。


1. **ルールベースの正規表現マッチング**:
   - カナと英字の表記ゆれを正規表現で対応付ける。
   - 例えば、「株式会社」「CO., LTD.」「Corporation」「Ltd.」などのパターンを正規化して比較する。
   - メリット: 高速で、特定のパターンに強い。
   - デメリット: ルールのメンテナンスが必要で、新しいパターンへの対応が遅れる可能性がある。

2. **辞書ベースの正規化**:
   - カナと英字の対応表を辞書として持ち、会社名を正規化する。
   - 例えば、「ソニー」と「Sony」などの対応を辞書に登録し、比較時に正規化して一致を確認する。
   - メリット: ルールベースより柔軟で、新しいパターンにも対応しやすい。
   - デメリット: 辞の計3コストが高くなる可能性がある。

4. **自然言語処理 (NLP) モデルの利用**:
   - 事前学習されたNLPモデルを使用して、カナと英字の表記ゆれを解消する。
   - 例えば、BERTやword2vecなどのモデルを使用して、意味的な類似性に基づいて一致を判断する。
   - メリット: 高度な柔軟性と精度。
   - デ4リット: 訓練データと計算リソースが必要。

5. **音声認識ベースのマッチング**:
   - 会社名の音声表現を基に一致を判断する。
   - 例えば、カナと英字の音声的な類似性を計算し、一致を確認する。
   - メリット: 特に日本語と英語の音声表現の違いを解消しやすい。
  
  
5. **生成AIベースのマッチング**:
   - 生成AIに会社名のペアを渡して、同じ会社かどうかチェックさせる。
   - 例えば、GPT-4o を用いて、会社名を比較させるようなプロンプトを作成し、一致度を0-5点で出す。
   - メリット: プロンプトの準備のみで事前に必要な処理やデータがない。
   - デメリット: 計算コストがかかるのと精度が安定しない可能性がある。
規化を試し、必要に応じてより高度な方法を導入するのが一般的です。

## 総合チェック (生成AIによるアプローチ)

In [109]:
# system メッセージの定義
system_message = """
会社名のペアが与えられた際に、表記の違いがあっても同じ会社かどうかを判断し、結果をJSON形式で出力してください。JSONのスキーマは、company_name1, company_name2, similarity_score, description として。考慮するべき違いには、スペース、文字の幅、言語、および略称が含まれますが、他にも様々な表記のパターンが存在することを念頭に置いてください。それぞれの比較について、以下のフォーマットを使用してください：

- 会社名1: <company1>
- 会社名2: <company2>
- 比較結果 (1-5):
- 説明:

比較結果のスコアリングは以下の通りです：
- 5: 絶対に一致している
- 1: 絶対に一致していない
- 2-4: 一致の可能性があるが、自信度によりスコアリング

### 例:

#### 1. スペースの有無
- 会社名1: 九州電子 株式会社
- 会社名2: 九州電子株式会社
- 比較結果: 5
- 説明: 「九州電子」と「株式会社」の間にスペースがあるかないかの違いだけであり、これは一般的な表記の揺れです。

#### 2. 半角全角の違い
- 会社名1: 合同会社All Right
- 会社名2: 合同会社Ａｌｌ　Ｒｉｇｈｔ
- 比較結果: 5
- 説明: 半角と全角文字の違いだけであり、会社の同一性には影響しません。

#### 3. カナ/英字の違い
- 会社名1: 大和ハウス工業 株式会社
- 会社名2: DAIWA HOUSE INDUSTRY CO., LTD.
- 比較結果: 5
- 説明: 日本語名と英語名で表記されているだけであり、国際的な企業では一般的です。

#### 4. 株式会社の有無
- 会社名1: T&Dリスクソリューションズ株式会社
- 会社名2: T&Dリスクソリューションズ
- 比較結果: 5
- 説明: 「株式会社」の有無は法的なデザインエータであり、会社の同一性には影響しません。

#### 5. 略称
- 会社名1: 株式会社福島中央ﾃﾚﾋﾞ
- 会社名2: fct.jp
- 比較結果: 5
- 説明: 一つは正式名称であり、もう一つは会社のドメイン名を使った略称です。

#### 6. その他のバリエーション
- 会社名1: 株式会社M-TEC
- 会社名2: 株式会社MｰTEC本社･工場
- 比較結果: 5
- 説明: 二つ目の名前には本社と工場の情報が含まれていますが、どちらも同じ企業を指しています。

#### 7. 曖昧な例
- 会社名1: 株式会社ヤマト
- 会社名2: ヤマト運輸株式会社
- 比較結果: 4
- 説明: 株式会社ヤマトとヤマト運輸株式会社は関連がある可能性がありますが、正式な名称として異なる場合もあるため、完全に一致しているかは不明です。

- 会社名1: NECソリューションイノベータ株式会社
- 会社名2: NECソリューションズ
- 比較結果: 3
- 説明: NECソリューションイノベータ株式会社とNECソリューションズは関連がある可能性がありますが、同一の会社であるかどうかは完全には確信が持てません。

- 会社名1: 株式会社みずほ銀行
- 会社名2: みずほフィナンシャルグループ
- 比較結果: 3
- 説明: 株式会社みずほ銀行とみずほフィナンシャルグループは関連会社である可能性が高いですが、同一の会社とは限りません。

- 会社名1: パナソニック株式会社
- 会社名2: パナソニックシステムソリューションズジャパン株式会社
- 比較結果: 2
- 説明: パナソニック株式会社とパナソニックシステムソリューションズジャパン株式会社は同じ企業グループ内の別会社である可能性が高いですが、同一の会社ではない可能性があります。

#### 8. 一致しない例
- 会社名1: 東京電力株式会社
- 会社名2: 中部電力株式会社
- 比較結果: 1
- 説明: 東京電力株式会社と中部電力株式会社は別々の企業であり、提供するサービスや所在地も異なります。

- 会社名1: 株式会社三菱
- 会社名2: 三菱商事株式会社
- 比較結果: 1
- 説明: 株式会社三菱と三菱商事株式会社は異なる会社であり、それぞれ異なる事業分野を持っています。

Based on these examples, please evaluate the following pair and output the result in JSON format with the following schema:

    "company_name1": "{company1}",
    "company_name2": "{company2}",
    "similarity_score": <score>,
    "description": "<explanation>"

"""

In [122]:
from openai import AzureOpenAI
import json
import os
import time

In [123]:
os.environ["AZURE_OPENAI_API_KEY"] = <Azure OpenAI の API Key>
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://<Azure OpenAI のサービス名>.openai.azure.com/'
os.environ["AZURE_DEPLOYMENT_NAME"] = <Azure OpenAI でデプロイした GPT-4o のデプロイ名>

In [124]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2023-12-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)


### GPT-4o による検証

In [160]:
# 会社名のペアを入力してスコアと説明を取得する関数
def evaluate_company_names(company1, company2):
    user_message = f"""
    - 会社名1: {company1}
    - 会社名2: {company2}
    """
    
    start_time = time.time()  # 開始時間を記録
    
    response = client.chat.completions.create(
        model=os.environ["AZURE_DEPLOYMENT_NAME"],
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        response_format={ "type": "json_object" },
    )

    result = response.choices[0].message.content.strip()
    result_json = json.loads(result)

    end_time = time.time()  # 終了時間を記録
    processing_time = end_time - start_time  # 処理時間を計算

    print(f"処理時間: {processing_time}秒")
    
    return result_json

In [161]:
# テストする会社名のペア
company1 = "三菱総研"
company2 = "MRI"

# 結果の取得
result = evaluate_company_names(company1, company2)

処理時間: 2.0334107875823975秒


In [163]:
result["similarity_score"]

4